# Replication: Nguyen *et al*, 2021

## Introduction

This notebook attempts to reproduce the following paper (which already uses the [PPMI](http://ppmi-info.org) dataset):

<div class="alert alert-block alert-success">
Nguyen KP, et al. <a href=https://doi:10.1016/j.parkreldis.2021.02.026>Predicting Parkinson's disease trajectory using clinical and neuroimaging baseline measures.</a> Parkinsonism Relat Disord. 2021;85:44-51. 
</div>

This study uses data from 82 PD subjects with rs-fMRI and MDS-UPDRS total score, encompassing both motor and non-motor symptomatology at the same visit. Of these 82 subjects, 53 subjects also had scores available at year 1 after imaging, 45 at year 2, and 33 at year 4. 

The fMRI data were acquired at resting-state on 3T scanners with the same acquisition parameters. Acquisition parameters are described below (table extracted from the original paper supplementary materials).

<img src="images/acquisition.png" width=800/>

The demographics parameters for the PD patients were as follows (table extracted from the paper):

<img src="images/demographics.png" width=800/>

The main goal of this paper is to use imaging features extracted from rs-fMRI data and demographic features to train machine learning models to predict MDS-UPDRS scores of PD patients.

Imaging features includes fractional Amplitude of Low Frequency Fluctuations (fALFF) and Regional Homogeneity (ReHo) averaged for different Regions of Interest (ROI) of the brain extracted from different atlases: [100-ROI Schaefer functional brain parcellation](https://doi.org/10.1093/cercor/bhx179), modified with an additional 35 striatal and cerebellar ROIs, 197-ROI and 444-ROI versions of the [Bootstrap Analysis of Stable Clusters (BASC197) atlas](https://doi.org/10.1016/j.neuroimage.2010.02.082).

Different machine learning models were compared: ElasticNet regression, Support Vector Machine (SVM) with a linear kernel, Random Forest with a decision tree kernel, and Gradient Boosting with a decision tree kernel. 

An unbiased random search was conducted to optimize the hyperparameters of each model, including regularization strength and learning rate. To determine the best-performing parcellation, hyperparameter, and model combination for each target, a rigorous nested cross-validation approach was applied, with leave-one-out cross-validation (LOOCV) as the outer loop and 10-fold cross-validation as the inner loop.

ReHo features explained 30.4%, 45.3%, 47.1%, and 25.5% of the variance in baseline, year 1, year 2, and year 4 MDS-UPDRS score, respectively. fALFF features explained 24.2%, 55.8%, 46.3%, and 15.2% of the variance in baseline, year 1, year 2, and year 4 MDS-UPDRS score, respectively. Results were significant at p = 0.001 (false discovery rate-corrected) at all timepoints except year 4, which was significant at p = 0.05.

Results are displayed below (table extracted from the original paper). 
<img src="images/results.png" width=800/>

## Initial setup

We first initialize the notebook cache and install dependencies:

In [1]:
import livingpark_utils

utils = livingpark_utils.LivingParkUtils()
#utils.notebook_init()

# import warnings filter
from warnings import simplefilter
# ignore all future warnings
simplefilter(action='ignore', category=FutureWarning)
simplefilter(action='ignore', category=UserWarning)
simplefilter(action='ignore', category=RuntimeWarning)

## PPMI cohort preparation

We will build a PPMI cohort that matches the one used in the original study (Table 1) as closely as possible. Our cohort will be built directly from PPMI Study Data files so that it can be replicated and updated whenever necessary.

### Study data download

We will start by downloading the PPMI Study Data files required to build our cohort: 

* Participant status (Parkinson's disease, healthy control, etc.)
* Demographics
* Age at visit
* Clinical/cognitive assessment results:
    * Montreal Cognitive Assessment (MoCA)
    * Unified Parkinson's Disease Rating Scale (UPDRS) Parts I, II and III
    * Geriatric Depression Scale (GDS)
    * Hoehn-Yahr stage

We will use the LivingPark utils library to download these files from the notebook. If files are already present in the notebook cache, they won't be downloaded again. Otherwise, a PPMI username and password are required to obtain the files. New PPMI accounts can be requested [here](http://ppmi-info.org).

In [2]:
from nguyenetal.constants import (
    FILENAME_PARTICIPANT_STATUS,
    FILENAME_DEMOGRAPHICS,
    FILENAME_PD_HISTORY,
    FILENAME_SOCIO,
    FILENAME_AGE,
    FILENAME_MOCA,
    FILENAME_UPDRS1A,
    FILENAME_UPDRS1B,
    FILENAME_UPDRS2,
    FILENAME_UPDRS3,
    FILENAME_UPDRS4,
    FILENAME_GDS,
    FILENAME_FMRI_INFO,
    FILENAME_FMRI_INFO_ZIP,
    FILENAME_FMRI_METADATA
)

from nguyenetal.constants import (
    COL_PAT_ID,
    COL_VISIT_TYPE,
    COL_STATUS,
    COL_PD_STATE,
    COL_AGE,
    COL_SEX,
    COL_EDUCATION,
    COL_UPDRS3,
    COL_UPDRS1A,
    COL_UPDRS1B,
    COL_UPDRS1,
    COL_UPDRS2,
    COL_UPDRS4,
    COL_MOCA,
)

from nguyenetal.constants import (
    COL_DATE_INFO,
    COL_DATE_BIRTH,
    COL_DATE_PD,
    FIELD_STRENGTH,
    STATUS_PD
    
)

from nguyenetal.constants import (
    COL_IMAGING_PROTOCOL,
    COLS_DATE, 
    IDA_STATUS_MAP,
    IDA_COLNAME_MAP,
    IDA_VISIT_MAP,
    STATUS_MED,
)

from nguyenetal.nguyenetal import (
    load_ppmi_csv,
    get_fMRI_cohort,
    mean_impute
)

from functools import reduce
import pandas as pd
import datetime as dt

In [3]:
from livingpark_utils.download.ppmi import Downloader

required_files = [
    FILENAME_PARTICIPANT_STATUS,
    FILENAME_PD_HISTORY,
    FILENAME_DEMOGRAPHICS,
    FILENAME_SOCIO,
    FILENAME_AGE,
    FILENAME_MOCA,
    FILENAME_UPDRS1A,
    FILENAME_UPDRS1B,
    FILENAME_UPDRS2,
    FILENAME_UPDRS3,
    FILENAME_UPDRS4,
    FILENAME_GDS
]

downloader = livingpark_utils.download.ppmi.Downloader(utils.study_files_dir)
utils.get_study_files(required_files, default=downloader)

## ASK TO BUILD A PPMI DOWNLOADER FOR FMRI CSV 

Download skipped: No missing files!


### Participants with fMRI data available

The main cohort contains 82 participants from either the `Parkinson's Disease` or the `GenCohort PD` cohort of PPMI. These 82 participants have rs-fMRI and outcome scores at the same visit available. MDS-UPDRS scores included the Part III Motor Examination conducted on-medication. Off-medication scores were not used due to unavailability for over half of the subjects and because examinations are more practically conducted on-medication in the clinic. 

We selected participants that:
* belonged to the selected cohort
* had fMRI scans with same acquisition parameters as those used in the paper.

In [4]:
print(f"=============== fMRI cohort ===============")
df_status = load_ppmi_csv(utils, FILENAME_PARTICIPANT_STATUS)
df_fMRI_subset = get_fMRI_cohort(utils)

# cohort composition: number of PD patients/healthy controls
print(
    df_status.loc[
        df_status[COL_PAT_ID].isin(df_fMRI_subset[COL_PAT_ID]), COL_STATUS
    ].value_counts()
)

=============== fMRI cohort ===============
Using fMRI info file: /Users/egermani/Documents/nguyen-etal-2021/inputs/study_files/fMRI_info.csv
Dropping 258 subjects with non-integer IDs
COHORT_DEFINITION
Parkinson's Disease    118
Name: count, dtype: int64


Then, we load/compute and merge all the required clinical/cognitive measures:
* UPDRS Part I
* UPDRS Part II
* UPDRS Part III
* UPDRS Part IV
* MoCA
* GDS Score

Missing values are imputed with the mean across the entire dataset, except for the UPDRS Part III score (handled below).

There are two files associated with UPDRS Part I (IA: Complex behaviors; IB: Partipant questionnaire). We use the sum of the total score in each file.

In [5]:
cols_for_merge = [COL_PAT_ID, COL_DATE_INFO, COL_VISIT_TYPE]

# Load necessary files
df_updrs1a = load_ppmi_csv(utils, FILENAME_UPDRS1A, convert_int = [COL_UPDRS1A], cols_to_impute=COL_UPDRS1A)
df_updrs1b = load_ppmi_csv(utils, FILENAME_UPDRS1B, convert_int = [COL_UPDRS1B], cols_to_impute=COL_UPDRS1B)
df_updrs2 = load_ppmi_csv(utils, FILENAME_UPDRS2, convert_int = [COL_UPDRS2], cols_to_impute=COL_UPDRS2)
df_updrs3 = load_ppmi_csv(utils, FILENAME_UPDRS3, convert_int = [COL_UPDRS3])
df_updrs4 = load_ppmi_csv(utils, FILENAME_UPDRS4, convert_int = [COL_UPDRS4], cols_to_impute=COL_UPDRS4)

df_moca = load_ppmi_csv(utils, FILENAME_MOCA, convert_int = [COL_MOCA])
df_gds = load_ppmi_csv(utils, FILENAME_GDS)

# Sum UPDRS IA and IB scores
df_updrs1 = df_updrs1a.merge(df_updrs1b, on=cols_for_merge)
df_updrs1[COL_UPDRS1] = df_updrs1.loc[:, [COL_UPDRS1A, COL_UPDRS1B]].sum(axis="columns")

# Drop unused UPDRSIII scores (ON medication)
df_updrs3 = df_updrs3.drop(df_updrs3.index[(df_updrs3['PAG_NAME'] == 'NUPDRS3') & \
                         (df_updrs3['EVENT_ID'].isin(['V04', 'V06', 'V08', 'V10', 'V12']))])

# Select UPDRS columns to merge
df_updrs1 = df_updrs1.loc[:, cols_for_merge + [COL_UPDRS1]]
df_updrs2 = df_updrs2.loc[:, cols_for_merge + [COL_UPDRS2]]
df_updrs3 = df_updrs3.loc[:, cols_for_merge + [COL_UPDRS3, 'NHY', 'PAG_NAME', 'PDSTATE']]
df_updrs4 = df_updrs4.loc[:, cols_for_merge + [COL_UPDRS4]]

# Compute GDS total score 
gds_cols = df_gds.columns[['GDS' in strcol for strcol in df_gds.columns]].tolist()
df_gds['GDS_TOTAL'] = df_gds[gds_cols].sum(axis=1)
df_gds = df_gds.loc[:, cols_for_merge + ['GDS_TOTAL']]

# Select MOCA columns to merge
df_moca = df_moca.loc[:, cols_for_merge + [COL_MOCA]]

# Merge 
df_assessments_all = reduce(
    lambda df1, df2: df1.merge(df2, on=cols_for_merge, how="outer"),
    [df_updrs2, df_updrs3, df_updrs1, df_updrs4, df_moca, df_gds],
).drop_duplicates()

# Compute TOTAL UPDRS SCORE 
updrs_cols = [COL_UPDRS1, COL_UPDRS2, COL_UPDRS3, COL_UPDRS4]
df_assessments_all['UPDRS_TOT'] = df_assessments_all[updrs_cols].sum(axis=1)
   
# Only keep cohort participants
df_cohort_assessments = df_assessments_all.loc[
    df_assessments_all[COL_PAT_ID].isin(df_fMRI_subset[COL_PAT_ID])]

# Drop participants that don't have UPDRS III Score
df_cohort_assessments = df_cohort_assessments.dropna(subset=['NP3TOT'])

Only participants with outcome score (UPDRS) and rs-fMRI data at the same visit were used, so we filter both datasets to keep only participants sessions that have both. 

In [6]:
df_fMRI_cohort = pd.DataFrame()
for i in range(len(df_cohort_assessments)):
    df_fMRI_cohort = pd.concat([df_fMRI_cohort, 
                        df_fMRI_subset[df_fMRI_subset[COL_PAT_ID] == df_cohort_assessments.iloc[i][COL_PAT_ID]]\
                        [df_fMRI_subset[COL_VISIT_TYPE] == df_cohort_assessments.iloc[i][COL_VISIT_TYPE]]]
                        )
    
df_scores_cohort = pd.DataFrame()
for i in range(len(df_fMRI_subset)):
    df_scores_cohort = pd.concat([df_scores_cohort, 
                    df_cohort_assessments[df_cohort_assessments[COL_PAT_ID] == df_fMRI_subset.iloc[i][COL_PAT_ID]]\
                    [df_cohort_assessments[COL_VISIT_TYPE] == df_fMRI_subset.iloc[i][COL_VISIT_TYPE]]]
                    )

In [7]:
fMRI_cols_to_include = ['PATNO', 'Sex','COHORT_DEFINITION','EVENT_ID', 'INFODT', 'Age', 
                        'Modality', 'Description', 'Imaging Protocol', 'Image ID']
scores_cols_to_include = ['PATNO', 'EVENT_ID', 'NP2PTOT', 'NP3TOT', 'NP1RTOT+NP1PTOT',
       'NP4TOT', 'NHY','MCATOT', 'GDS_TOTAL', 'UPDRS_TOT']

df_fMRI_cohort = df_fMRI_cohort.loc[:, fMRI_cols_to_include]
df_scores_cohort = df_scores_cohort.loc[:, scores_cols_to_include]

# Merge important columns from both datasets
df_global_cohort = df_fMRI_cohort.merge(df_scores_cohort, on=[COL_PAT_ID, COL_VISIT_TYPE])
df_global_cohort = df_global_cohort.sort_values(by=['PATNO','INFODT'])

### Baseline cohort

For the training, authors used the first scan & outcome score available for each participant. 

In [8]:
df_global_cohort_baseline = df_global_cohort.drop_duplicates(subset=COL_PAT_ID)
df_global_cohort_baseline = df_global_cohort_baseline[
    df_global_cohort_baseline[COL_DATE_INFO] < pd.Timestamp(2020, 1, 1, 12)
    ] # Removed due to the date of the study

In [9]:
print('Number of participants selected using papers informations:', len(df_global_cohort_baseline))

Number of participants selected using papers informations: 102


### Prediction cohort 

In the paper, authors are trying to predict UPDRS scores at baseline (same session as fMRI data), 1 year after, 2 years after and 4 years after. 

In [10]:
# DF with outcome scores for every participants
df_global_cohort_pred = df_cohort_assessments[df_cohort_assessments\
                                              [COL_PAT_ID].isin(df_global_cohort_baseline[COL_PAT_ID].tolist())]

# Filter by date due to the date of publication of the paper. 
df_global_cohort_pred = df_global_cohort_pred[df_global_cohort_pred[COL_DATE_INFO] < pd.Timestamp(2020, 1, 1, 12)]

# Event taken as Baseline
df_global_cohort_pred['BASELINE_EV'] = [df_global_cohort_baseline[COL_VISIT_TYPE]\
                [df_global_cohort_baseline[COL_PAT_ID] == df_global_cohort_pred[COL_PAT_ID].iloc[i]].iloc[0] \
                             for i in range(len(df_global_cohort_pred))]

In [11]:
eq_1year = {
    'BL':'V04',
    'ST':'V04',
    'V04':'V06',
    'V06':'V08',
    'V08':'V10',
    'V10':'V12'
}

eq_2year = {
    'BL':'V06',
    'ST':'V06',
    'V04':'V08',
    'V06':'V10',
    'V08':'V12',
    'V10':'V13'
}

eq_4year = {
    'BL':'V10',
    'ST':'V10',
    'V04':'V12',
    'V06':'V13',
    'V08':'V14',
    'V10':'V15'
}

In [12]:
df_global_cohort_pred['1Y_EVENT'] = [eq_1year[b] for b in df_global_cohort_pred['BASELINE_EV'].tolist()]
df_global_cohort_pred['2Y_EVENT'] = [eq_2year[b] for b in df_global_cohort_pred['BASELINE_EV'].tolist()]
df_global_cohort_pred['4Y_EVENT'] = [eq_4year[b] for b in df_global_cohort_pred['BASELINE_EV'].tolist()]

In [13]:
df_global_1y = df_global_cohort_pred[
        df_global_cohort_pred[COL_VISIT_TYPE]==df_global_cohort_pred['1Y_EVENT']
    ].drop_duplicates(subset=COL_PAT_ID)

df_global_2y = df_global_cohort_pred[
        df_global_cohort_pred[COL_VISIT_TYPE]==df_global_cohort_pred['2Y_EVENT']
    ].drop_duplicates(subset=COL_PAT_ID)

df_global_4y = df_global_cohort_pred[
        df_global_cohort_pred[COL_VISIT_TYPE]==df_global_cohort_pred['4Y_EVENT']
    ].drop_duplicates(subset=COL_PAT_ID)

In [14]:
print('Number of participants for 1y cohort: ', len(df_global_1y))
print('Number of participants for 2y cohort: ', len(df_global_2y))
print('Number of participants for 4y cohort: ', len(df_global_4y))

Number of participants for 1y cohort:  63
Number of participants for 2y cohort:  71
Number of participants for 4y cohort:  53


### Final cohort

Using the information that we had in the paper, we were able to select 105 participants against 82 mentioned in the paper. We create a demographics table similar to the one in the original paper to verify our cohort. 

In [15]:
# Summary of BASELINE cohort
import numpy as np

def to_1_decimal_str(f):
    return str(round(f, 1))

# Load necessary study files
df_pd_history = load_ppmi_csv(utils, FILENAME_PD_HISTORY)
df_demographics = load_ppmi_csv(utils, FILENAME_DEMOGRAPHICS)
df_socio = load_ppmi_csv(utils, FILENAME_SOCIO)

# Necessary columns for demographic file
cols_demo = ['PATNO', 'BIRTHDT', 'RAWHITE', 'HISPLAT', 'RABLACK', 'RAASIAN', 'HANDED']

df_summary = df_global_cohort_baseline.merge(
    df_demographics[cols_demo],
    on=[COL_PAT_ID],
)

# Necessary columns for PD file
cols_PD = ['PATNO','PDDXDT']

df_summary = df_summary.merge(
    df_pd_history[cols_PD],
    on=[COL_PAT_ID],
)

# Necessary columns for social file 
cols_socio = ['PATNO','EDUCYRS']

df_summary = df_summary.merge(
    df_socio[cols_socio].drop_duplicates(subset=[COL_PAT_ID]),
    on=[COL_PAT_ID],
)

# Conversion of dates & binaries
df_summary['Days since diagnosis, baseline'] = (
        df_summary[COL_DATE_INFO] - df_summary[COL_DATE_PD]) / np.timedelta64(1, 'D')
df_summary['Days since diagnosis, baseline'].astype(int)
baseline_days_to_diagnosis = df_summary[['PATNO','Days since diagnosis, baseline']]

df_summary['Sex'] = [1 if sex=='M' else 0 for sex in df_summary['Sex'].tolist()]
df_summary['HANDED'] = [1 if h==1 else 0 for h in df_summary['HANDED'].tolist()]

df_summary['UPDRS_TOT_TIMEPOINT'] = df_summary['UPDRS_TOT']
updrs_baseline = df_summary[['PATNO','UPDRS_TOT']]

# Drop unused columns
df_summary = df_summary.drop(['PATNO','BIRTHDT', 'PDDXDT','Modality', 'INFODT', 
                               'EVENT_ID', 'Description',
                               'COHORT_DEFINITION', 'Imaging Protocol'], axis = 1)

# Create table 
index={
        "RAWHITE":"% Caucasian",
        "RABLACK":"% African-American",
        "RAASIAN":"% Asian",
        'HISPLAT':"% Hispanic",
        'Sex': "% Male",
        'HANDED': '% right-handed',
        'Age': "Mean age, years",
        'EDUCYRS': 'Mean years of education',
        'Days since diagnosis, baseline':"Mean disease duration at baseline, days",
        'UPDRS_TOT' : "Mean MDS-UPDRS at baseline",
        'UPDRS_TOT_TIMEPOINT': 'Mean MDS-UPDRS at timepoint',
        'MCATOT': "Mean MoCA at baseline",
        'GDS_TOTAL': "Mean GDS at Baseline",
        'NHY': "Mean Hoehn-Yahr stage"
    }


df_summary = df_summary[list(index.keys())]

for keys in list(index.keys())[:6]:
    df_summary[keys] = df_summary[keys] * 100 

df_summary_means = df_summary.mean().tolist()
df_summary_stds = df_summary.std().tolist()

df_summary_means = [to_1_decimal_str(mean) for mean in df_summary_means] 
df_summary_stds = [" ± " + to_1_decimal_str(std) if i > 5 else '' for i, std in enumerate(df_summary_stds)] 
df_summary_baseline = pd.DataFrame([df_summary_means, df_summary_stds], columns = df_summary.columns)
df_summary_baseline = (df_summary_baseline.iloc[0] + df_summary_baseline.iloc[1]).T

df_summary_baseline = df_summary_baseline.rename(index=index)
df_summary_baseline.loc['Mean MDS-UPDRS at timepoint'] = '-'

In [16]:
# Summary of Prediction cohort
df_age = load_ppmi_csv(utils, FILENAME_AGE)

# Create table 
index={
        "RAWHITE":"% Caucasian",
        "RABLACK":"% African-American",
        "RAASIAN":"% Asian",
        'HISPLAT':"% Hispanic",
        'Sex': "% Male",
        'HANDED': '% right-handed',
        'AGE_AT_VISIT': "Mean age, years",
        'EDUCYRS': 'Mean years of education',
        'Days since diagnosis, baseline':"Mean disease duration at baseline, days",
        'UPDRS_TOT_BL':"Mean MDS-UPDRS at baseline",
        'UPDRS_TOT' : "Mean MDS-UPDRS at timepoint",
        'MCATOT': "Mean MoCA at baseline",
        'GDS_TOTAL': "Mean GDS at Baseline",
        'NHY': "Mean Hoehn-Yahr stage"
    }

df_summary_pred = []
df_summary_pred_values = []

for i, pred_df in enumerate([df_global_1y, df_global_2y, df_global_4y]):
    # Find back sex columns in baseline cohort
    pred_df['Sex'] = [
        df_global_cohort_baseline['Sex'][df_global_cohort_baseline[COL_PAT_ID]==sub].iloc[0] \
        for sub in pred_df[COL_PAT_ID].tolist()]
    
    pred_df = pred_df.merge(df_age, on=[COL_PAT_ID, COL_VISIT_TYPE])
    
    # Necessary columns for demographic file
    cols_demo = ['PATNO', 'BIRTHDT', 'RAWHITE', 'HISPLAT', 'RABLACK', 'RAASIAN', 'HANDED']

    df_summary_pred.append(pred_df.merge(
        df_demographics[cols_demo],
        on=[COL_PAT_ID],
    ))

    # Necessary columns for PD file
    cols_PD = ['PATNO','PDDXDT']

    df_summary_pred[i] = df_summary_pred[i].merge(
        df_pd_history[cols_PD],
        on=[COL_PAT_ID],
    )

    # Necessary columns for social file 
    cols_socio = ['PATNO','EDUCYRS']

    df_summary_pred[i] = df_summary_pred[i].merge(
        df_socio[cols_socio].drop_duplicates(subset=[COL_PAT_ID]),
        on=[COL_PAT_ID],
    )
    
    # Conversion to binaries and dates
    df_summary_pred[i] = df_summary_pred[i].drop_duplicates(subset=[COL_PAT_ID])
    
    df_summary_pred[i]['Days since diagnosis, baseline'] = df_summary[
        "Days since diagnosis, baseline"][baseline_days_to_diagnosis[COL_PAT_ID].isin(
        pred_df[COL_PAT_ID].tolist())].tolist()
    
    df_summary_pred[i]['UPDRS_TOT_BL'] = updrs_baseline[
        "UPDRS_TOT"][updrs_baseline[COL_PAT_ID].isin(
        pred_df[COL_PAT_ID].tolist())].tolist()
    
    df_summary_pred[i]['Sex'] = [1 if sex=='M' else 0 for sex in df_summary_pred[i]['Sex'].tolist()]
    df_summary_pred[i]['HANDED'] = [1 if h==1 else 0 for h in df_summary_pred[i]['HANDED'].tolist()]

    # Drop unused columns
    df_summary_pred[i] = df_summary_pred[i].drop(['PATNO','BIRTHDT', 'PDDXDT', 'INFODT', 
                                   'EVENT_ID'], axis = 1)
    # Keep only necessary columns
    df_summary_pred[i] = df_summary_pred[i][list(index.keys())]
    
    # Transform to percentage when necessary (6 first values)
    for keys in list(index.keys())[:6]:
        df_summary_pred[i][keys] = df_summary_pred[i][keys] * 100 

    df_summary_means = df_summary_pred[i].mean().tolist()
    df_summary_stds = df_summary_pred[i].std().tolist()

    df_summary_means = [to_1_decimal_str(mean) for mean in df_summary_means] 
    df_summary_stds = [" ± " + to_1_decimal_str(std) if i > 5 else '' \
                       for i, std in enumerate(df_summary_stds)] 
    
    df_summary_pred_values.append(pd.DataFrame([df_summary_means, df_summary_stds], 
                                               columns = df_summary_pred[i].columns))
    
    df_summary_pred_values[i] = (df_summary_pred_values[i].iloc[0] + df_summary_pred_values[i].iloc[1]).T
    df_summary_pred_values[i] = df_summary_pred_values[i].rename(index=index)

In [17]:
df_allyears_summary = pd.DataFrame(columns = [('Baseline', 'Original'), ('Baseline', 'Replication'),
                                              ('Year 1', 'Original'), ('Year 1', 'Replication'), 
                                              ('Year 2', 'Original'), ('Year 2', 'Replication'), 
                                              ('Year 4', 'Original'), ('Year 4', 'Replication')])

df_allyears_summary[('Baseline', 'Original')]=['95.1',
 '2.4',
 '3.7',
 '1.2',
 '67.0',
 '89.0',
 '62.1 ± 9.8',
 '15.6 ± 3.0',
 '770 ± 565',
 '33.9 ± 15.8',
 '-',
 '26.7 ± 2.8',
 '5.4 ± 1.4',
 '1.8 ± 0.5']

df_allyears_summary[('Year 1', 'Original')] = ['94.4',
 '1.9',
 '5.6',
 '0',
 '68.5',
 '85.2',
 '61.9 ± 10.3',
 '15.1 ± 3.2',
 '808 ± 576',
 '38.0 ± 20.9',
 '39.2 ± 21.6',
 '26.9 ± 3.2',
 '5.4 ± 1.6',
 '1.8 ± 0.5']

df_allyears_summary[('Year 2', 'Original')] = ['97.8',
 '0',
 '4.4',
 '0',
 '82.2',
 '88.9',
 '63.6 ± 9.2',
 '15.1 ± 3.3',
 '771 ± 506',
 '40.2 ± 18.2',
 '40.9 ± 18.5',
 '26.7 ± 3.5',
 '5.4 ± 1.2',
 '1.8 ± 0.5']

df_allyears_summary[('Year 4', 'Original')] = ['97.0',
 '0',
 '3.0',
 '0',
 '75.8',
 '87.9',
 '59.5 ± 11.0',
 '15.0 ± 3.4',
 '532 ± 346',
 '34.9 ± 15.7',
 '35.9 ± 16.5',
 '27.5 ± 2.3',
 '5.4 ± 1.7',
 '1.7 ± 0.5']

df_allyears_summary[('Baseline', 'Replication')] = df_summary_baseline.tolist()
df_allyears_summary[('Year 1', 'Replication')] = df_summary_pred_values[0].tolist()
df_allyears_summary[('Year 2', 'Replication')] = df_summary_pred_values[1].tolist()
df_allyears_summary[('Year 4', 'Replication')] = df_summary_pred_values[2].tolist()

df_allyears_summary.index = df_summary_baseline.index

df_allyears_summary.loc['Number of subject'] = [82, len(df_global_cohort_baseline), 53, len(df_global_1y), 
                                      45, len(df_global_2y), 33, len(df_global_4y)]

df_allyears_summary.columns = pd.MultiIndex.from_tuples(df_allyears_summary.columns)
df_allyears_summary

Baseline                 \
                                            Original    Replication   
% Caucasian                                     95.1           95.1   
% African-American                               2.4            2.0   
% Asian                                          3.7            2.9   
% Hispanic                                       1.2            1.0   
% Male                                          67.0           66.7   
% right-handed                                  89.0           89.2   
Mean age, years                           62.1 ± 9.8     62.1 ± 9.5   
Mean years of education                   15.6 ± 3.0     15.6 ± 2.8   
Mean disease duration at baseline, days    770 ± 565  887.9 ± 618.4   
Mean MDS-UPDRS at baseline               33.9 ± 15.8    34.5 ± 15.6   
Mean MDS-UPDRS at timepoint                        -              -   
Mean MoCA at baseline                     26.7 ± 2.8     26.5 ± 3.0   
Mean GDS at Baseline                       5.4 ± 1.4      5.4 ± 1.4   
Mean Hoehn-Yahr stage                      1.8 ± 0.5      1.7 ± 0.5   
Number of subject                                 82            102   

                                              Year 1                  \
                                            Original     Replication   
% Caucasian                                     94.4            95.2   
% African-American                               1.9             1.6   
% Asian                                          5.6             3.2   
% Hispanic                                         0             1.6   
% Male                                          68.5            65.1   
% right-handed                                  85.2            87.3   
Mean age, years                          61.9 ± 10.3      62.5 ± 9.7   
Mean years of education                   15.1 ± 3.2      15.1 ± 2.6   
Mean disease duration at baseline, days    808 ± 576  1017.5 ± 627.4   
Mean MDS-UPDRS at baseline               38.0 ± 20.9     33.8 ± 15.3   
Mean MDS-UPDRS at timepoint              39.2 ± 21.6     39.7 ± 21.9   
Mean MoCA at baseline                     26.9 ± 3.2      27.0 ± 2.9   
Mean GDS at Baseline                       5.4 ± 1.6       5.5 ± 1.7   
Mean Hoehn-Yahr stage                      1.8 ± 0.5       1.7 ± 0.5   
Number of subject                                 53              63   

                                              Year 2                 \
                                            Original    Replication   
% Caucasian                                     97.8           97.2   
% African-American                                 0            0.0   
% Asian                                          4.4            2.8   
% Hispanic                                         0            1.4   
% Male                                          82.2           76.1   
% right-handed                                  88.9           87.3   
Mean age, years                           63.6 ± 9.2     63.7 ± 9.8   
Mean years of education                   15.1 ± 3.3     15.4 ± 2.7   
Mean disease duration at baseline, days    771 ± 506  954.5 ± 605.9   
Mean MDS-UPDRS at baseline               40.2 ± 18.2    34.8 ± 15.4   
Mean MDS-UPDRS at timepoint              40.9 ± 18.5    42.3 ± 20.6   
Mean MoCA at baseline                     26.7 ± 3.5     26.9 ± 2.4   
Mean GDS at Baseline                       5.4 ± 1.2      5.6 ± 1.4   
Mean Hoehn-Yahr stage                      1.8 ± 0.5      1.9 ± 0.5   
Number of subject                                 45             71   

                                              Year 4                 
                                            Original    Replication  
% Caucasian                                     97.0           98.1  
% African-American                                 0            0.0  
% Asian                                          3.0            1.9  
% Hispanic                                         0 

The values obtained in this table are basically similar to those in the paper, except in terms of number of participants. The original baseline cohort was composed of 82 participants compared to 102 in our case. Year 1, 2 and 4 cohorts also exhibit a larger number of participants than in the original paper. 

Mean values of demographics and clinical features are similar to those of the original paper. 

To obtain the same number of participants in each cohort, we will randomly sample the same number of participants as those used in the paper and perform the replication on these participants.

## Download T1 data

fMRI data are associated with anatomical T1 data, often necessary for preprocessing. 

We searched for the T1 data acquired during the same session as the fMRI ones for the baseline cohort. We chose the first one each time.

In [18]:
df_mri = load_ppmi_csv(utils, 'MRI_info.csv', from_ida_search=True)

df_mri_cohort_baseline = df_mri.merge(df_global_cohort_baseline[[COL_PAT_ID, COL_VISIT_TYPE]], 
                      on = [COL_PAT_ID, COL_VISIT_TYPE])

df_mri_cohort_baseline = df_mri_cohort_baseline.sort_values(by=[COL_PAT_ID,'Description'])
df_mri_cohort_baseline = df_mri_cohort_baseline.drop_duplicates(subset=[COL_PAT_ID])

print('Number of cohort participants with T1 scans:', len(df_mri_cohort_baseline))

Dropping 3 subjects with non-integer IDs
Number of cohort participants with T1 scans: 102


## Extracting features for machine learning models

In the paper, authors reported having used different clinical and demographic features along with the radiomic data to train the models. 
These included:
* **Clinical features**: disease duration, symptom duration, dominant symptom side, Geriatric Depression Scale (GDS), Montreal Cognitive Assessment (MoCA), and presence of tremor, rigidity, or postural instability at baseline. Baseline MDS-UPDRS score was also included as a confounding variable when training models to predict future outcomes. 
* **Demographic features**: age, sex, ethnicity, race, handedness, and years of education.

<div class="alert alert-block alert-danger">
    Dominant side is not included on PD features in the PPMI database, so we removed it from the used features.
</div>

In [22]:
# BASELINE COHORT
# Load necessary study files
df_pd_history = load_ppmi_csv(utils, FILENAME_PD_HISTORY)
df_demographics = load_ppmi_csv(utils, FILENAME_DEMOGRAPHICS)
df_socio = load_ppmi_csv(utils, FILENAME_SOCIO)

df_features = df_global_cohort_baseline[['PATNO', 'Sex', 'Age', 'MCATOT', 'GDS_TOTAL']]

# Necessary columns for demographic file
cols_demo = ['PATNO', 'RAWHITE', 'HISPLAT', 'RAINDALS','RABLACK', 'RAASIAN', 
             'RAHAWOPI', 'RANOS','HANDED']

df_features = df_features.merge(
    df_demographics[cols_demo],
    on=[COL_PAT_ID],
)

# Necessary columns for PD file
cols_PD = ['PATNO','INFODT','SXDT', 'PDDXDT', 'DXTREMOR', 'DXRIGID', 
           'DXBRADY', 'DXPOSINS']

df_features = df_features.merge(
    df_pd_history[cols_PD],
    on=[COL_PAT_ID],
)

# Necessary columns for social file 
cols_socio = ['PATNO','EDUCYRS']

df_features = df_features.merge(
    df_socio[cols_socio].drop_duplicates(subset=[COL_PAT_ID]),
    on=[COL_PAT_ID],
)

# Transform columns to binaries or dates
df_features['Sex'] = [1 if sex=='M' else 0 for sex in df_features['Sex'].tolist()]

df_features['INFODT'] = pd.to_datetime(df_features['INFODT'], format='%m-%y')
df_features['PDDXDT'] = pd.to_datetime(df_features['PDDXDT'], format='%m-%y')

# Manage dates with SXDT columns (M/Y to M-Y)
list_symptom_date = []
for i, row in df_features.iterrows():
    date = row['SXDT']
    m = int(date.split('/')[0])
    y = int(date.split('/')[1])
    list_symptom_date += [dt.datetime.strptime('{:02d}-{}'.format(int(m), int(y)), '%m-%Y')]

df_features['SXDT'] = list_symptom_date

# Days bw diagnosis and visit
df_features['V-DXDT'] = (
        df_features[COL_DATE_INFO] - df_features[COL_DATE_PD]) / np.timedelta64(1, 'D')

# Days bw symptoms and visit
df_features['V-SXDT'] = (
        df_features[COL_DATE_INFO] - df_features['SXDT']) / np.timedelta64(1, 'D')

df_features = df_features.drop(['INFODT', 'SXDT', 'PDDXDT'],axis=1)
df_features = df_features.set_index('PATNO')

In [23]:
# How to deal with NaNs ? 

def impute_mean(df, col_name, is_int=False):
    '''
    Replace NaNs with mean.
    '''
    mean_value = df[col_name].mean()
    if is_int:
        mean_value = int(mean_value)
    df[col_name].fillna(value=mean_value, inplace=True)
    
    return df

df_features = impute_mean(df_features, 'MCATOT', True)
df_features = impute_mean(df_features, 'GDS_TOTAL', True)

In [24]:
df_features

,Sex,Age,MCATOT,GDS_TOTAL,RAWHITE,HISPLAT,RAINDALS,RABLACK,RAASIAN,RAHAWOPI,RANOS,HANDED,DXTREMOR,DXRIGID,DXBRADY,DXPOSINS,EDUCYRS,V-DXDT,V-SXDT
PATNO,,,,,,,,,,,,,,,,,,,
3107.0,1,71.7,23.0,6.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,0.0,16.0,59.0,1277.0
3108.0,0,51.8,30.0,5.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,0.0,15.0,0.0,911.0
3113.0,0,61.3,24.0,8.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,1.0,1.0,0.0,16.0,0.0,730.0
3116.0,1,66.0,15.0,8.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,1.0,1.0,1.0,1.0,18.0,669.0,1095.0
3118.0,1,64.4,24.0,4.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,0.0,14.0,31.0,214.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
52678.0,1,68.8,26.0,5.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,0.0,19.0,273.0,1522.0
53060.0,1,68.1,26.0,4.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,0.0,16.0,854.0,1188.0
55395.0,0,66.9,26.0,3.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,0.0,23.0,1340.0,1522.0


## Not included in Notebook: 

In [45]:
sub_list = [3107,
 3108,
 3113,
 3116,
 3123,
 3124,
 3125,
 3126,
 3127,
 3128,
 3130,
 3134,
 3327,
 3332,
 3352,
 3354,
 3359,
 3360,
 3364,
 3365,
 3366,
 3367,
 3371,
 3372,
 3373,
 3375,
 3378,
 3380,
 3383,
 3385,
 3386,
 3387,
 3392,
 3552,
 3557,
 3567,
 3574,
 3575,
 3577,
 3586,
 3587,
 3588,
 3589,
 3591,
 3592,
 3593,
 3760,
 3800,
 3808,
 3814,
 3815,
 3818,
 3819,
 3822,
 3823,
 3825,
 3826,
 3828,
 3829,
 3830,
 3831,
 3832,
 3834,
 3838,
 3870,
 4020,
 4024,
 4026,
 4030,
 4034,
 4035,
 40366,
 4038,
 40533,
 50485,
 50901,
 51632,
 51731,
 52678,
 53060,
 55395,
 70463]

ses_list = ['5/15/2013',
 '4/24/2013',
 '7/17/2013',
 '11/14/2012',
 '6/19/2013',
 '7/17/2013',
 '7/10/2013',
 '9/18/2013',
 '1/25/2017',
 '9/19/2013',
 '11/16/2012',
 '4/22/2013',
 '11/29/2012',
 '4/23/2013',
 '3/13/2013',
 '3/28/2013',
 '9/12/2013',
 '7/31/2013',
 '6/25/2013',
 '10/09/2013',
 '9/11/2013',
 '8/15/2013',
 '2/01/2013',
 '2/27/2013',
 '8/01/2013',
 '7/05/2013',
 '7/03/2013',
 '8/21/2013',
 '10/10/2012',
 '12/06/2012',
 '12/20/2012',
 '1/10/2013',
 '4/29/2013',
 '1/07/2013',
 '2/24/2015',
 '6/29/2015',
 '10/30/2013',
 '10/31/2012',
 '12/07/2015',
 '8/18/2014',
 '9/17/2014',
 '9/16/2016',
 '1/25/2013',
 '3/08/2013',
 '4/15/2013',
 '3/27/2013',
 '1/23/2013',
 '3/19/2013',
 '11/19/2013',
 '12/10/2013',
 '11/03/2015',
 '4/08/2014',
 '4/09/2013',
 '5/28/2013',
 '6/04/2013',
 '7/30/2013',
 '8/20/2013',
 '9/17/2013',
 '10/01/2013',
 '11/26/2013',
 '10/29/2013',
 '12/03/2013',
 '1/14/2014',
 '4/01/2014',
 '12/17/2012',
 '2/16/2016',
 '8/02/2016',
 '8/30/2016',
 '1/03/2013',
 '4/02/2013',
 '3/13/2013',
 '9/24/2014',
 '4/01/2013',
 '6/21/2017',
 '3/09/2016',
 '12/17/2014',
 '11/18/2015',
 '9/29/2015',
 '11/18/2015',
 '11/09/2016',
 '1/05/2017',
 '8/31/2017']

In [46]:
sub_list_cohort = df_global_cohort['PATNO'].tolist()
print('Missing subjects:', [i for i in sub_list if i not in sub_list_cohort])

Missing subjects: []


In [47]:
df_global_cohort_baseline['INCLUDED'] = [sub in sub_list for sub in df_global_cohort_baseline['PATNO']]

In [ ]:
df_global_cohort_baseline = df_global_cohort_baseline[df_global_cohort_baseline['EVENT_ID']!='V10']

In [ ]:
ses_ymj = [ses.split('/')[-1] + '-0' + ses.split('/')[0] + '-'+ses.split('/')[1] if len(ses.split('/')[0])==1 else ses.split('/')[-1] + '-' + ses.split('/')[0] + '-' + ses.split('/')[1] for ses in ses_list]
df_global_cohort_baseline['SES_YMJ'] = 0
df_global_cohort_baseline['SES_YMJ'][df_global_cohort_baseline['INCLUDED']==True] = ses_ymj 

In [ ]:
df_global_cohort_baseline['Correct session'] = [
    True if str(df_global_cohort_baseline['INFODT'].tolist()[i]
               )[:10] == df_global_cohort_baseline['SES_YMJ'].tolist()[i] else False for i in range(
        len(df_global_cohort_baseline))]

In [26]:
df_global_cohort_baseline = df_global_cohort_baseline[df_global_cohort_baseline['INCLUDED']==True]
#.columns#[df_global_cohort['Correct session']==False][df_global_cohort['INCLUDED']==True]

,PATNO,Sex,COHORT_DEFINITION,EVENT_ID,INFODT,Age,Modality,Description,Imaging Protocol,Image ID,Archive Date,NP2PTOT,NP3TOT,NP1RTOT+NP1PTOT,NP4TOT,NHY,MCATOT,GDS_TOTAL,UPDRS_TOT,INCLUDED
0,3107.0,M,Parkinson's Disease,V06,2013-05-15,71.7,fMRI,ep2d_RESTING_STATE,Field Strength=3.0;TE=25.0;Manufacturer=SIEMEN...,378215,6/26/2013,5.0,6.0,8.0,0.0,2.0,23.0,6.0,19.0,True
2,3108.0,F,Parkinson's Disease,V06,2013-04-24,51.8,fMRI,ep2d_RESTING_STATE,Field Strength=3.0;TE=25.0;Manufacturer=SIEMEN...,378223,6/26/2013,3.0,15.0,3.0,0.0,2.0,30.0,5.0,21.0,True
4,3113.0,F,Parkinson's Disease,V06,2013-07-17,61.3,fMRI,ep2d_RESTING_STATE,Field Strength=3.0;TE=25.0;Manufacturer=SIEMEN...,393649,10/08/2013,12.0,23.0,3.0,0.0,2.0,24.0,8.0,38.0,True
5,3116.0,M,Parkinson's Disease,V04,2012-11-14,66.0,fMRI,ep2d_RESTING_STATE,Field Strength=3.0;TE=25.0;Manufacturer=SIEMEN...,366137,4/09/2013,14.0,33.0,16.0,0.0,2.0,15.0,8.0,63.0,True
7,3118.0,M,Parkinson's Disease,V10,2016-02-24,64.4,fMRI,ep2d_RESTING_STATE,Field Strength=3.0;TE=25.0;Manufacturer=SIEMEN...,665280,3/30/2016,13.0,7.0,8.0,0.0,2.0,24.0,4.0,28.0,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
174,52678.0,M,Parkinson's Disease,BL,2015-11-18,68.8,fMRI,ep2d_RESTING_STATE,Field Strength=3.0;TE=25.0;Manufacturer=SIEMEN...,581173,12/15/2015,8.0,42.0,2.0,0.0,2.0,NaN,5.0,52.0,True
179,53060.0,M,Parkinson's Disease,BL,2016-11-09,68.1,fMRI,ep2d_RESTING_STATE,Field Strength=3.0;TE=25.0;Manufacturer=SIEMEN...,831138,3/20/2017,0.0,4.0,13.0,0.0,1.0,NaN,4.0,17.0,True
180,55395.0,F,Parkinson's Disease,BL,2017-01-05,66.9,fMRI,ep2d_bold_rest,Field Strength=3.0;TE=25.0;Manufacturer=SIEMEN...,845691,5/01/2017,10.0,40.0,10.0,0.0,3.0,NaN,3.0,60.0,True
181,55468.0,F,Parkinson's Disease,BL,2019-06-19,51.7,fMRI,ep2d_RESTING_STATE,Field Strength=3.0;TE=25.0;Manufacturer=SIEMEN...,1224327,9/09/2019,2.0,11.0,3.0,8.0,2.0,NaN,3.0,24.0,False
